In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

I've also made a file called `summary.txt`


In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
1121, Brock St S, Whitby, Ontario,
L1N 4M1
+13063515344 (Mobile)
mohan.khilariwal@gmail.com
www.linkedin.com/in/
mohankhilariwal (LinkedIn)
www.mohankhilariwal.blogspot.com
(Blog)
Top Skills
Amazon Web Services (AWS)
Google Cloud Platform (GCP)
Generative AI
Languages
English (Full Professional)
Hindi (Native or Bilingual)
Certifications
AI and ML for Business Leaders
Hadoop Foundation Level 1
The Data Science Course 2019:
Complete Data Science Bootcamp
Generative AI for Business: Driving
Growth and Competitive Advantage
Python 101 for DataScience
Honors-Awards
Honorary Advisor - Heritage
Restoration Ecosystem
Zenith Award - 2023 Inspiring AI &
Analytics Leader
Mohan Khilariwal
Innovator | GenAI/AI/ML Practitioner, consultant | Fractional CTO/
CAIO | Digital Transformation Leader| 3AI Thought Leader
Whitby, Ontario, Canada
Summary
I am seeking senior leadership roles in Generative AI (GenAI)
and AI/ML—within Canada or remotely—where I can leverage my
extensive background in

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Mohan Khilariwal"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

"You are acting as Mohan Khilariwal. You are answering questions on Mohan Khilariwal's website, particularly questions related to Mohan Khilariwal's career, background, skills and experience. Your responsibility is to represent Mohan Khilariwal for interactions on the website as faithfully as possible. You are given a summary of Mohan Khilariwal's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Mohan Khilariwal. I'm a senior Leader, software engineer and data scientist. I'm originally from Delhi, India, but I moved to Toronto in 2024.\nI love all foods, particularly Indian food, but strangely I'm repelled by almost all forms of meat. I'm not allergic, I just hate the thought that I am eating an animal for the sake of taste buds! I make an exception for eggs and dairy products though - co

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [14]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [15]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

'Yes, I hold a patent for a Driver’s Gaze Detection System, which is aimed at enhancing automotive safety. The patent applications are filed in India (India Patent Application Number 202311046856) and the United States (US Patent Application Number 18/369,229, USPTO Patent # 12540924). This innovation reflects my commitment to leveraging technology for impactful solutions. If you have any further questions about my work or experience, feel free to ask!'

In [18]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is acceptable. The agent appropriately answers the question about the patent from the user, and then additionally includes the patent numbers. They also ask if the user has any further questions, which is engaging.')

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [20]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
This response is not acceptable because it's written in Pig Latin. While it may have been an attempt to be funny, it's unprofessional. Please be serious and engaging without using humour.
